When thinking in Foundational Models we should think on a model trained on generic visual patterns that can be transferred to new tasks without task-specific training. In Earth Observation (EO) it means, it doesnt know about land cover, physics, vegetation types. It only know textures, shapes, contrast. IT is powerful but dangerous.

SAM is zero-shot ; object agnostic ; scale-agnostic ; spectrally blind. Therefore, it creates the perfect tension: it segments beautifully, but it understands nothing environmentally. It segment baesd on texture/shape/contrast alone ignoring spectal signatures (NDVI for vegetation) or physical context (reef vs sandbar distinction in turbid water)

Concept: The Segment Anything Model (SAM) has "seen" 11 billion images, but it doesn't know what a "tree" is. It only knows what an "object" is (texture, edge, contrast).

Instead of loading a downloaded image lets swithc to cloud straming keeping the same areas Hulhudhoo Maldives for coastal/reef segmentation; and Hyde Park/London for urban green space. 

LEts use pystac-client + planetary_computer + stackstac (already in the container)

Do these polygons correspond to meaningful geographic objects?

Could you calculate area from these?

Would a policymaker trust these shapes?
SAM Porposes objects, We test whether those objects are environmentally meaningful.

In [4]:
import leafmap
import os
from samgeo import SamGeo
import pystac_client
import planetary_computer
import rioxarray

In [2]:
# Bounding boxes (W, S, E, N) in lon/lat (EPSG:4326)

AOI = {
    "Hulhudhoo_Maldives": (73.20, 3.82, 73.26, 3.88),
    "HydePark_London": (-0.19, 51.50, -0.15, 51.52),
}

AOI


{'Hulhudhoo_Maldives': (73.2, 3.82, 73.26, 3.88),
 'HydePark_London': (-0.19, 51.5, -0.15, 51.52)}

In [5]:
catalog = pystac_client.Client.open(
    "https://planetarycomputer.microsoft.com/api/stac/v1",
    modifier=planetary_computer.sign_inplace,
)

print("Connected to Planetary Computer STAC")


Connected to Planetary Computer STAC


In [6]:
site = "HydePark_London"
bbox = AOI[site]

search = catalog.search(
    collections=["sentinel-2-l2a"],
    bbox=bbox,
    datetime="2021-04-01/2021-05-15",
    query={"eo:cloud_cover": {"lt": 10}},
)

items = list(search.get_items())
len(items)


2

In [7]:
item = items[0]
item.id, item.datetime, item.properties.get("eo:cloud_cover")


('S2B_MSIL2A_20210423T105619_R094_T30UXC_20210525T005413',
 datetime.datetime(2021, 4, 23, 10, 56, 19, 24000, tzinfo=tzutc()),
 1.047312)

In [11]:
import folium

# Map centered on Hyde Park
m = folium.Map(location=[51.51, -0.17], zoom_start=14, tiles="OpenStreetMap")

# AOI bbox = (W, S, E, N)
west, south, east, north = bbox

# Draw AOI rectangle
folium.Rectangle(
    bounds=[(south, west), (north, east)],
    color="yellow",
    fill=False,
    weight=3,
).add_to(m)

# Draw STAC item footprint (GeoJSON)
folium.GeoJson(
    data=item.geometry,
    name="Sentinel-2 scene footprint",
).add_to(m)

folium.LayerControl().add_to(m)
m
